# Import von WIAG-Daten in FactGrid
Formatiere CSV-Daten aus WIAG als V1-Statements für QuickSteps  
siehe [Einführung in QuickStatements](https://www.wikidata.org/wiki/Help:QuickStatements)
und die [Properties des FactGrid](https://database.factgrid.de/wiki/FactGrid:Directory_of_Properties)

Änderungen

- *2023-12-11* Verwende die Gruppe des höchsten erreichten Amtes in der englischen Beschreibung und für P165

Inhalt:

- [neue Objekte in FactGrid anlegen](#neue-Objekte-in-FactGrid-anlegen)

In [9]:
using CSV, DataFrames, Dates

In [10]:
ENV["COLUMNS"] = 120

120

In [11]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [12]:
output_path = input_path

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

## neue Objekte in FactGrid anlegen
Es wird davon ausgegangen, dass die betreffenden Personen noch nicht in FactGrid vorhanden sind. Die Spalte "FactGrid_ID" sollte also für alle Einträge leer sein.

Daten aus WIAG einlesen: Quelle Export "CSV Personendaten"

In [81]:
input_file = "WIAG-Köln_2023-12-18.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\WIAG-Köln_2023-12-18.csv"

In [82]:
df_person_all = CSV.read(input_path_file, DataFrame);
nrow(df_person_all)

1168

Betrachte nur Neu-Zugänge (keine FactGrid-ID)

In [83]:
df_person_in = subset(df_person_all, :FactGrid_ID => ByRow(ismissing));
nrow(df_person_in)

956

In [84]:
sort!(df_person_in, :corpus);

Lies die Amtsdaten ein (zunächst für P1018)

In [17]:
input_file = "WIAG-Köln-roles_2023-12-18.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\WIAG-Köln-roles_2023-12-18.csv"

In [18]:
df_role_all = CSV.read(input_path_file, DataFrame);
nrow(df_role_all)

4563

Definiere Funktionen:  
Lege die zwei Rollen/Amtszeiten fest, die für die Beschreibung verwendet werden.  
Priorisiere nach Gruppe der Rolle (siehe `role_group_rank_list`) und dann nach Amtszeit (jüngste zuerst)

In [21]:
function describe_role(role)
    inst_or_dioc = !ismissing(role[:institution]) ? role[:institution] : role[:diocese]    
    
    date_info = ""
    if !ismissing(role[:date_begin]) && !ismissing(role[:date_end])
        date_info = role[:date_begin] * "-" * role[:date_end]
    elseif !ismissing(role[:date_begin])
        date_info = role[:date_begin]
    elseif !ismissing(role[:date_end])
        date_info = "bis " * role[:date_end]
    end
    
    description = role[:name]
    if !ismissing(inst_or_dioc)
        description *= " " * inst_or_dioc
    end
    if date_info != ""
        description *= " " * date_info
    end
    
    return description    
end

describe_role (generic function with 1 method)

In [3]:
role_group_rank_list = [
    "Q648236", # Leiter (Erz-)diözese (Altes Reich)
    "Q648232", # Domdignitär Altes Reich
    "Q648233", # Klosterangehöriger mit Leitungsamt
    "Q646226", # Domkleriker Altes Reich
]

4-element Vector{String}:
 "Q648236"
 "Q648232"
 "Q648233"
 "Q646226"

In [4]:
"""
    lt_role_group_fq_id(a, b, role_group_rank_list)

compare `a` and `b` based on `role_group_rank_list`
"""
function less_than_role_group_fq_id(a, b, role_group_rank_list)
    if ismissing(a)
        return false
    end
    if ismissing(b)
        return true
    end
    
    lt = false
    for rg_prio in role_group_rank_list
        if a == rg_prio && b != rg_prio
            lt = true
            break
        end
        if a != rg_prio && b == rg_prio
            lt = false
            break
        end
    end
    
    return lt        
end

lt_rg_fq_id(a, b) = less_than_role_group_fq_id(a, b, role_group_rank_list)

lt_rg_fq_id (generic function with 1 method)

In [40]:
function description_by_role(df)
    N_ROLE_4_DESCRIPTION = 2
    df_s = sort(df, [:date_sort_key], rev = true)
    df_s = sort(df_s, [:role_group_fq_id], lt = lt_rg_fq_id, alg = MergeSort)
    description_list = String[]
    for row in eachrow(df_s)
        push!(description_list, describe_role(row))
    end
    head_list = first(unique(description_list), N_ROLE_4_DESCRIPTION)
    return join(head_list, ", ")
end

description_by_role (generic function with 1 method)

In [48]:
dfg_role_all = groupby(df_role_all, [:person_id]);
df_role_description = combine(dfg_role_all, description_by_role);
nrow(df_role_description)

1168

In [42]:
df_role_description[200:205, :]

Row,person_id,x1
,String31,String
1,WIAG-Pers-CANON-19190-001,Domherr Domstift Köln 1255-1277
2,WIAG-Pers-CANON-19191-001,Domherr Domstift Köln 1255/1256-1288
3,WIAG-Pers-CANON-19189-001,"Propst Kollegiatstift St. Cassius, Bonn 1243, Propst Kollegiatstift St. Georg, Köln 1231-1241"
4,WIAG-Pers-CANON-19192-001,Domherr Domstift Köln 1256-1292
5,WIAG-Pers-CANON-18523-001,Domherr Domstift Köln 1256-1275
6,WIAG-Pers-CANON-51991-001,Domherr Domstift Köln 1256-1257


Test
Karl Erzherzog von Österreich, WIAG-Pers-EPISCGatz-20417-001

In [43]:
subset(df_role_description, :person_id => ByRow(isequal("WIAG-Pers-EPISCGatz-20417-001")))

Row,person_id,x1
,String31,String
1,WIAG-Pers-EPISCGatz-20417-001,"Bischof Bistum Brixen 1613-1624, Bischof Bistum Breslau 1608-1624"


In [87]:
columns = [
    :person_id => :person_id,
    :x1 => :summary_roles,
]

2-element Vector{Pair{Symbol, Symbol}}:
 :person_id => :person_id
        :x1 => :summary_roles

In [88]:
df_person = innerjoin(df_person_in, select(df_role_description, columns), on = :id => :person_id);
nrow(df_person)

956

In [89]:
df_person[200:204, [:id, :displayname, :summary_roles]]

Row,id,displayname,summary_roles
,String31,String,String
1,WIAG-Pers-CANON-18532-001,Florenz von Wevelinghoven,Domherr Domstift Köln 1266/1270-1318
2,WIAG-Pers-CANON-18397-001,Hermann von Rennenberg,"Domsubdekan Domstift Köln 1304-1318, Domherr Domstift Köln 1268-1318"
3,WIAG-Pers-CANON-18533-001,Ludwig von Neuenahr,Domherr Domstift Köln 1269
4,WIAG-Pers-CANON-18406-001,Albert von Hammerstein,"Domchorbischof Domstift Köln 1307-1316, Domherr Domstift Köln 1270"
5,WIAG-Pers-CANON-51995-001,Arnold von Rennenberg,Domherr Domstift Köln 1270-1296


Wähle für die Liste der Rollen die relevanten Spalten aus.

In [65]:
columns = [
    :person_id => :person_id,
    :role_group_fq_id => :P1018
]

2-element Vector{Pair{Symbol, Symbol}}:
        :person_id => :person_id
 :role_group_fq_id => :P1018

In [66]:
df_role = select(df_role_all, columns);

Es werden die Ämter für P1018 eingetragen, deren Gruppen über die entsprechenden FactGrid-IDs hier ausgewählt werden.

In [67]:
relevant_role_group_fq_id = [
    "Q648236", # Leiter (Erz-)diözese (Altes Reich)
    "Q648232", # Domdignitär Altes Reich    
];

In [62]:
is_relevant(s) = !ismissing(s) && s in relevant_role_group_fq_id

is_relevant (generic function with 1 method)

In [68]:
df_role = subset(df_role, :P1018 => ByRow(is_relevant));
nrow(df_role)

1176

Das deutsche Beschreibungsfeld soll Lebensdaten mit der Zusammenfassung der Amtsdaten enthalten.

In [90]:
join_komma(a, b) = join((a, b), ", ")
transform!(df_person, [:biographical_dates, :summary_roles] => ByRow(join_komma) => :description_de);

Das FactGrid arbeitet für Wikipedia nicht mit URL-codierten Adressen.
*2023-12-13* WIAG gibt direkt unkodierte URLs aus, daher sind die folgenden Schritte auskommentiert

In [20]:
# unescape_not_mg(s) = ismissing(s) ? s : URIs.unescapeuri(s)

In [21]:
# transform!(df_person, :Wikipedia => ByRow(unescape_not_mg) => :Wikipedia);

In [91]:
columns = [:givenname, :prefix, :familyname, :Wikipedia]
dropmissing(df_person, :Wikipedia)[5:9, columns]

Row,givenname,prefix,familyname,Wikipedia
,String,String15?,String?,String
1,Gottfried,de,Fontibus,Gottfried_von_Fontaines
2,Wilhelm,von,Berg,Wilhelm_I._(Berg)
3,Wedekind,von,Holte,Wedekind_von_Holte
4,Wilhelm,von,Jülich,Wilhelm_der_Jüngere_(Jülich)
5,Wikbold,von,Lohn,Wikbold_von_Lohn


Benenne die Spalten um entsprechend den Konventionen des FactGrid  
*2023-12-14* `career` wird jetzt mehrfach anhand der Amtsdaten eingetragen (siehe unten)

In [93]:
columns = [
    :displayname => :Lde,
    :description_de => :Dde,
    :date_of_birth => :P77,
    :date_of_death => :P38,
    :GND_ID => :P76,
    :GSN => :P472,
    :id => :P601,
    :Wikidata_ID => :Swikidatawiki,
    :Wikipedia => :Sdewiki
];

In [94]:
rename!(df_person, columns);

Kopiere das Label in Deutsch für die anderen Sprachen

In [95]:
df_person.Len = df_person.Lde;
df_person.Lfr = df_person.Lde;
df_person.Les = df_person.Lde;

Erstelle eine Beschreibung für Englisch aus Lebensdaten und der Gruppe des höchsten Amtes (Spalte 'career_en')

In [96]:
join_komma(a, b) = join([a, b], ", ")

join_komma (generic function with 1 method)

In [97]:
transform!(df_person, [:biographical_dates, :career_en] => ByRow(join_komma) => :Den);

In [99]:
df_person[111:114, [:P601, :givenname, :prefix, :familyname, :Len, :Den]]

Row,P601,givenname,prefix,familyname,Len,Den
,String31,String,String15?,String?,String,String
1,WIAG-Pers-CANON-18383-001,Konrad,von,Rennenberg,Konrad von Rennenberg,"+ 1264, Dignitary of a cathedral chapter"
2,WIAG-Pers-CANON-18392-001,Albert,von,Linnep,Albert von Linnep,"+ 1243, Cleric of the Cathedral chapter"
3,WIAG-Pers-CANON-18488-001,Dietrich,von,Kempenich,Dietrich von Kempenich,"~ 1222, Cleric of the Cathedral chapter"
4,WIAG-Pers-CANON-18423-001,Philipp,von,Altena-Isenberg,Philipp von Altena-Isenberg,"+ 1265?, Dignitary of a cathedral chapter"


Füge Daten ein, die für alle Personen gleich sind:
Mensch, Teil der Germania Sacra Forschungsdaten, männlich

In [100]:
insertcols!(df_person, 
    :P2 => "Q7",
    :P131 => "Q153178",
    :P154 => "Q18"
);    

Definiere Umwandlungsfunktionen  
*offen:* Datumsangaben sind noch zu besprechen, wegen der Verarbeitung von unscharften Zeitangaben.

In [101]:
fqs_string(s) = "\"" * string(s) * "\""

fqs_string (generic function with 1 method)

In [102]:
function create(out, row, col_list) 
    println(out, "CREATE")    
    for col in col_list
        println(out, "LAST\t", string(col), "\t", fqs_string(row[col]))        
    end
end

create (generic function with 1 method)

In [103]:
"""
    set_property_list (out, row, object, col_list, fmt_list)

    

# Arguments
- `out::IOStream`: output stream
- `row::DataFrameRow`: input data
- `object`: FactGrid identifier or "LAST"
- `col_list`: list of colums where data should be read from
- `fmr_list`: list of formatting functions (one for each column)
"""
function set_property_list(out, row, object, col_list, fmt_list)
    for (col, fmt) in zip(col_list, fmt_list)
        if (!ismissing(row[col]))
            println(out, object, "\t", string(col), "\t", fmt(row[col]))
        end
    end
end

set_property_list

Lies die Amtsdaten für einen bestimmten Domherren

In [104]:
"""
    set_role(df_role, id)

extract roles for person with `id`
"""
function set_role(out, df_role, id, object, property)
    property = Symbol(property)
    df_pr = subset(df_role, :person_id => ByRow(isequal(id)));
    df_pr = subset(df_pr, )
    fq_id_list = unique(df_pr[!, property])
    for fq_id in fq_id_list
        println(out, object, "\t", property, "\t", fq_id)
    end
    return nothing
end


set_role

Gib ausgewählte Elemente aus `df_person` aus. Falls es schon eine Datei mit gleichem Namen im angegebenen Verzeichnis gibt, wird die Datei überschrieben.

In [105]:
date_key = Dates.format(now(), "yyyy-mm-dd")

"2023-12-18"

In [106]:
output_file = "Insert_WIAG-Köln_" * date_key * "_FQ.csv"
output_path_file = joinpath(output_path, output_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\Insert_WIAG-Köln_2023-12-18_FQ.csv"

In [107]:
open(output_path_file, "w") do out_stream
    for row in eachrow(df_person)
        create(out_stream, row, [:Lde, :Len, :Lfr, :Les, :Dde, :Den])
        set_property_list(out_stream, row, "LAST", 
            [:P2, :P131, :P154, :P601, :P76, :P472, :Swikidatawiki, :Sdewiki], 
            [identity, identity, identity, fqs_string, fqs_string, fqs_string, fqs_string, fqs_string])
        set_role(out_stream, df_role, row[:P601], "LAST", :P1018)
    end        
end